# Homework 10: `SQL`

## Due Date:  Thursday, November 16th at 11:59 PM

You will create a database of the NASA polynomial coefficients for each specie.

**Please turn in your database with your `Jupyter` notebook!**

# Question 1: Convert XML to a SQL database

Create two tables named `LOW` and `HIGH`, each corresponding to data given for the low and high temperature range.
Each should have the following column names:

- `SPECIES_NAME`
- `TLOW`
- `THIGH`
- `COEFF_1`
- `COEFF_2`
- `COEFF_3`
- `COEFF_4`
- `COEFF_5`
- `COEFF_6`
- `COEFF_7`

Populate the tables using the XML file you created in last assignment. If you did not complete the last assignment, you may also use the `example_thermo.xml` file.

`TLOW` should refer to the temperature at the low range and `THIGH` should refer to the temperature at the high range.  For example, in the `LOW` table, $H$ would have `TLOW` at $200$ and `THIGH` at $1000$ and in the `HIGH` table, $H$ would have `TLOW` at $1000$ and `THIGH` at $3500$.

For both tables, `COEFF_1` through `COEFF_7` should be populated with the corresponding coefficients for the low temperature data and high temperature data.

In [1]:
from IPython.display import display

In [2]:
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import re
from copy import deepcopy
import numpy as np
import sqlite3
import pandas as pd

class PolynomialsParser:
    def __init__(self):
        self.data = None        
    
    def parse_raw(self, file_name, save2xml=False, new_file_name=None, i_start=5, lines=4):
        with open(file_name, 'r') as f:
            raw = f.read().strip()
        raw = raw.split('\n')
        species = [raw[i:i+lines] for i in range(i_start, len(raw), lines)][:-1]

        sci_number = re.compile('-?[0-9]+\.?[0-9]*E[+-]?[0-9]*')
        def find_sci_numbers(s):
            return [x for x in re.findall(sci_number, s)]
        
        def parse_specie(specie):
            name = specie[0].split()[0].strip()
            Ts = specie[0].split()[-4:-1]
            line_vals = [find_sci_numbers(s) for s in specie[1:]]
            coeff_high = line_vals[0] + line_vals[1][:2]
            coeff_low = line_vals[1][2:] + line_vals[2]
            return name, Ts, coeff_high, coeff_low
        
        data = dict()
        data['speciesArray'] = []
        data['speciesData'] = dict()
        
        for specie in species:
            name, Ts, coeff_high, coeff_low = parse_specie(specie)
            data['speciesArray'].append(name)
            data['speciesData'][name] = {'Ts':Ts, 'coeff_high':coeff_high, 'coeff_low':coeff_low}
        if save2xml:
            if new_file_name is None:
                new_file_name = file_name[:-4] + '.xml'
            self.save_xml(data, new_file_name)
            
        def to_float(data_s):
            data = deepcopy(data_s)
            for name, _data in data['speciesData'].items():
                _data['Ts'] = [float(s) for s in _data['Ts']]
                _data['coeff_high'] = [float(s) for s in _data['coeff_high']]
                _data['coeff_low'] = [float(s) for s in _data['coeff_low']]
            return data
        
        self.data = to_float(data)
        
        return self
    
    def save_xml(self, data, file_name):
        root = ET.Element('ctml')
        
        root.append(ET.Comment('phase gri30'))
        
        phase = ET.SubElement(root, 'phase', id='gri30')
        ET.SubElement(phase, 'speciesArray', datasrc='#species_data').text = ' '.join(data['speciesArray'])
        
        root.append(ET.Comment('species definitions'))
        
        def add_speciesData(speciesData, data, p0="100000.0"):
            for name in data['speciesArray']:
                _data = data['speciesData'][name]
                speciesData.append(ET.Comment('species {}'.format(name)))
                specie = ET.SubElement(speciesData, 'species', name=name)
                thermo = ET.SubElement(specie, 'thermo')
                low = ET.SubElement(thermo, 'NASA', Tmax=_data['Ts'][-1], Tmin=_data['Ts'][0], p0=p0)
                ET.SubElement(low, 'floatArray', name='coeffs', size=str(len(_data['coeff_low']))).\
                text = ', '.join(_data['coeff_low'])
                high = ET.SubElement(thermo, 'NASA', Tmax=_data['Ts'][1], Tmin=_data['Ts'][-1], p0=p0)
                ET.SubElement(high, 'floatArray', name='coeffs', size=str(len(_data['coeff_high']))).\
                text = ', '.join(_data['coeff_high'])
        
        speciesData = ET.SubElement(root, 'speciesData', id='species_data')
        add_speciesData(speciesData, data)
        
        tree = ET.ElementTree(root)

        xmlstr = BeautifulSoup(ET.tostring(root), 'xml').prettify()
        with open(file_name, 'w') as f:
            f.write(xmlstr)
            
        return self
    
    
    def parse_xml(self, file_name):
        raw = ET.parse(file_name).getroot()
        data = dict()
        data['speciesArray'] = raw.find('phase').find('speciesArray').text.strip().split()
        data['speciesData'] = dict()
        for specie in raw.find('speciesData').findall('species'):
            name = specie.attrib['name']
            data['speciesData'][name] = dict()
            data['speciesData'][name]['Ts'] = [0 for _ in range(3)]
            NASAs = specie.find('thermo').findall('NASA')
            i_high = np.argmax([float(NASA.attrib['Tmax']) for NASA in NASAs])
            data['speciesData'][name]['Ts'][0] = float(NASAs[1 - i_high].attrib['Tmin'])
            data['speciesData'][name]['Ts'][1] = float(NASAs[i_high].attrib['Tmax'])
            data['speciesData'][name]['Ts'][2] = float(NASAs[i_high].attrib['Tmin'])
            data['speciesData'][name]['coeff_high'] = [float(s) for s in NASAs[i_high].\
                                                       find('floatArray').text.strip().split(',')]
            data['speciesData'][name]['coeff_low'] = [float(s) for s in NASAs[1 - i_high].\
                                                       find('floatArray').text.strip().split(',')]
        self.data = data
        
        return self
    
    def xml2sql(self, xml_name, sql_name=None):
        if sql_name is None:
            sql_name = xml_name[:-4] + '.sqlite'
        self.parse_xml(xml_name)
        data = self.data
        db = sqlite3.connect(sql_name)
        cursor = db.cursor()
        cursor.execute('DROP TABLE IF EXISTS LOW')
        cursor.execute('DROP TABLE IF EXISTS HIGH')
        cursor.execute('''CREATE TABLE LOW (
                        SPECIES_NAME TEXT PRIMARY KEY NOT NULL,
                        TLOW FLOAT,
                        THIGH FLOAT,
                        COEFF_1 FLOAT,
                        COEFF_2 FLOAT,
                        COEFF_3 FLOAT,
                        COEFF_4 FLOAT,
                        COEFF_5 FLOAT,
                        COEFF_6 FLOAT,
                        COEFF_7 FLOAT)''')
        cursor.execute('''CREATE TABLE HIGH (
                        SPECIES_NAME TEXT PRIMARY KEY NOT NULL,
                        TLOW FLOAT,
                        THIGH FLOAT,
                        COEFF_1 FLOAT,
                        COEFF_2 FLOAT,
                        COEFF_3 FLOAT,
                        COEFF_4 FLOAT,
                        COEFF_5 FLOAT,
                        COEFF_6 FLOAT,
                        COEFF_7 FLOAT)''')
        for species in data['speciesArray']:
            _data = data['speciesData'][species]
            _low = [species, _data['Ts'][0], _data['Ts'][-1]] + _data['coeff_low']
            _high = [species, _data['Ts'][-1], _data['Ts'][1]] + _data['coeff_high']
            cursor.execute('''INSERT INTO LOW (SPECIES_NAME, TLOW, THIGH, COEFF_1, COEFF_2,
            COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', _low)
            cursor.execute('''INSERT INTO HIGH (SPECIES_NAME, TLOW, THIGH, COEFF_1, COEFF_2,
            COEFF_3, COEFF_4, COEFF_5, COEFF_6, COEFF_7)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', _high)
        db.commit()
        db.close()
        
        return self
    
    def viz_tables(self, sql_name):
        db = sqlite3.connect(sql_name)
        cursor = db.cursor()
        cols = ['SPECIES_NAME', 'TLOW', 'THIGH', 'COEFF_1', 'COEFF_2', 'COEFF_3', \
                'COEFF_4', 'COEFF_5', 'COEFF_6', 'COEFF_7']
        queries = ['''SELECT * FROM LOW''', '''SELECT * FROM HIGH''']
        qs = [cursor.execute(query).fetchall() for query in queries]
        dfs = [pd.DataFrame.from_items([(col_name, [col[i] for col in q]) \
                                        for i, col_name in enumerate(cols)]) for q in qs]
        db.close()
        return dfs

In [3]:
pp = PolynomialsParser().xml2sql('thermo.xml')
dfs = pp.viz_tables('thermo.sqlite')
for df in dfs:
    display(df)

,SPECIES_NAME,TLOW,THIGH,COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7
0,O,200.0,1000.0,3.168267,-3.279319e-03,6.643064e-06,-6.128066e-09,2.112660e-12,29122.259200,2.051933
1,O2,200.0,1000.0,3.782456,-2.996734e-03,9.847302e-06,-9.681295e-09,3.243728e-12,-1063.943560,3.657676
2,H,200.0,1000.0,2.500000,7.053328e-13,-1.995920e-15,2.300816e-18,-9.277323e-22,25473.659900,-0.446683
3,H2,200.0,1000.0,2.344331,7.980521e-03,-1.947815e-05,2.015721e-08,-7.376118e-12,-917.935173,0.683010
4,OH,200.0,1000.0,3.992015,-2.401318e-03,4.617938e-06,-3.881133e-09,1.364115e-12,3615.080560,-0.103925
5,H2O,200.0,1000.0,4.198641,-2.036434e-03,6.520402e-06,-5.487971e-09,1.771978e-12,-30293.726700,-0.849032
6,HO2,200.0,1000.0,4.301798,-4.749121e-03,2.115829e-05,-2.427639e-08,9.292251e-12,294.808040,3.716662
7,H2O2,200.0,1000.0,4.276113,-5.428224e-04,1.673357e-05,-2.157708e-08,8.624544e-12,-17702.582100,3.435051


,SPECIES_NAME,TLOW,THIGH,COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7
0,O,1000.0,3500.0,2.569421,-8.597411e-05,4.194846e-08,-1.001778e-11,1.228337e-15,29217.579100,4.784339
1,O2,1000.0,3500.0,3.282538,1.483088e-03,-7.579667e-07,2.094706e-10,-2.167178e-14,-1088.457720,5.453231
2,H,1000.0,3500.0,2.500000,-2.308430e-11,1.615619e-14,-4.735152e-18,4.981974e-22,25473.659900,-0.446683
3,H2,1000.0,3500.0,3.337279,-4.940247e-05,4.994568e-07,-1.795664e-10,2.002554e-14,-950.158922,-3.205023
4,OH,1000.0,3500.0,3.092888,5.484297e-04,1.265052e-07,-8.794616e-11,1.174124e-14,3858.657000,4.476696
5,H2O,1000.0,3500.0,3.033992,2.176918e-03,-1.640725e-07,-9.704199e-11,1.682010e-14,-30004.297100,4.966770
6,HO2,1000.0,3500.0,4.017211,2.239820e-03,-6.336581e-07,1.142464e-10,-1.079085e-14,111.856713,3.785102
7,H2O2,1000.0,3500.0,4.165003,4.908317e-03,-1.901392e-06,3.711860e-10,-2.879083e-14,-17861.787700,2.916157


# Question 2: `WHERE` Statements

1. Write a `Python` function `get_coeffs` that returns an array of 7 coefficients.  
   
   The function should take in two parameters: 1.) `species_name` and 2.) `temp_range`, an indicator variable ('low' or 'high') to indicate whether the coefficients should come from the low or high temperature range.  
   The function should use `SQL` commands and `WHERE` statements on the table you just created in Question 1 (rather than taking data from the XML directly).
```python
def get_coeffs(species_name, temp_range):
    ''' Fill in here'''
    return coeffs
```

2. Write a python function `get_species` that returns all species that have a temperature range above or below a given value. The function should take in two parameters: 1.) `temp` and 2.) `temp_range`, an indicator variable ('low' or 'high').

  When temp_range is 'low', we are looking for species with a temperature range lower than the given temperature, and for a 'high' temp_range, we want species with a temperature range higher than the given temperature.

  This exercise may be useful if different species have different `LOW` and `HIGH` ranges.

  And as before, you should accomplish this through `SQL` queries and where statements.

```python
def get_species(temp, temp_range):
    ''' Fill in here'''
    return coeffs
```

In [4]:
# 1
def get_coeffs(species_name, temp_range, sql_name='thermo.sqlite'):
    db = sqlite3.connect(sql_name)
    cursor = db.cursor()
    temp_range = temp_range.strip().upper()
    if temp_range not in ['LOW', 'HIGH']:
        raise ValueError('Invalid temp_range.')
    query = '''SELECT * FROM {} WHERE SPECIES_NAME == "{}"'''.format(temp_range, species_name)
    coeffs = cursor.execute(query).fetchall()[0][3:]
    db.close()
    return coeffs

In [5]:
print(get_coeffs('H', 'low'))
print(get_coeffs('O', 'high'))

(2.5, 7.05332819e-13, -1.99591964e-15, 2.30081632e-18, -9.27732332e-22, 25473.6599, -0.446682853)
(2.56942078, -8.59741137e-05, 4.19484589e-08, -1.00177799e-11, 1.22833691e-15, 29217.5791, 4.78433864)


In [6]:
# 2
def get_species(temp, temp_range, sql_name='thermo.sqlite'):
    db = sqlite3.connect(sql_name)
    cursor = db.cursor()
    temp_range = temp_range.strip().upper()
    if temp_range not in ['LOW', 'HIGH']:
        raise ValueError('Invalid temp_range.')
    if temp_range == 'LOW':
        query = '''SELECT SPECIES_NAME FROM HIGH WHERE THIGH < {}'''.format(temp)
    else:
        query = '''SELECT SPECIES_NAME FROM LOW WHERE TLOW > {}'''.format(temp)
    species = [s[0] for s in cursor.execute(query).fetchall()]
    db.close()
    return species

In [7]:
print(get_species(10000, 'low'))
print(get_species(50, 'high'))
print(get_species(500, 'low'))
print(get_species(500, 'high'))

['O', 'O2', 'H', 'H2', 'OH', 'H2O', 'HO2', 'H2O2']
['O', 'O2', 'H', 'H2', 'OH', 'H2O', 'HO2', 'H2O2']
[]
[]


# Question 3: `JOIN` STATEMENTS

Create a table named `ALL_TEMPS` that has the following columns:

- `SPECIES_NAME`
- `TEMP_LOW`
- `TEMP_HIGH`

This table should be created by joining the tables `LOW` and `HIGH` on the value `SPECIES_NAME`.

1. Write a `Python` function `get_range` that returns the range of temperatures for a given species_name.

The range should be computed within the `SQL` query (i.e. you should subtract within the `SELECT` statement in the `SQL` query).
```python
def get_range(species_name):
    '''Fill in here'''
    return range
```

Note that `TEMP_LOW` is the lowest temperature in the `LOW` range and `TEMP_HIGH` is the highest temperature in the `HIGH` range.

In [8]:
def add_ALL_TEMPS(sql_name='thermo.sqlite'):
    db = sqlite3.connect(sql_name)
    cursor = db.cursor()
    cursor.execute('DROP TABLE IF EXISTS ALL_TEMPS')
    cursor.execute('''CREATE TABLE ALL_TEMPS
    AS SELECT LOW.SPECIES_NAME, LOW.TLOW AS TEMP_LOW, HIGH.THIGH AS TEMP_HIGH
    FROM LOW
    INNER JOIN HIGH ON LOW.SPECIES_NAME = HIGH.SPECIES_NAME''')
    db.commit()
    db.close()

add_ALL_TEMPS()

In [9]:
def get_range(species_name, sql_name='thermo.sqlite'):
    db = sqlite3.connect(sql_name)
    cursor = db.cursor()
    T_range = cursor.execute('''SELECT TEMP_LOW, TEMP_HIGH FROM ALL_TEMPS WHERE SPECIES_NAME ==\
    "{}"'''.format(species_name)).fetchall()[0]
    db.close()
    return T_range

In [10]:
get_range('H')

(200.0, 3500.0)